### DUCKDB

DuckDB est une librairie de Python qui semblerait récupérer des meta-données tout en étant plus rapide que PostgreSQL, SQLite...

Cette librairie serait plus performante que PySpark (reconnu pour être moins rapide que la librairie Polars pour les fichiers de taille moyenne)

<br>Site officiel : https://duckdb.org/

Lien : https://www.youtube.com/watch?v=AjsB6lM2-zw

Exercice : https://robust-dinosaur-2ef.notion.site/DuckDB-Tutorial-Getting-started-for-beginners-b80bf0de8d6142d6979e78e59ffbbefe

Date : 30-03-2023

In [1]:
# %pip install duckdb

In [16]:
import pandas as pd
import glob # pour récupérer un certain nombre de fichiers
import time
import duckdb

In [17]:
# Connection sql
conn = duckdb.connect()

In [18]:
# Assignation d'un minuteur
cur_time = time.time()

In [19]:
# Concaténation de tous les fichiers CSV récupérés avec pandas 😮
df = pd.concat([pd.read_csv(f) for f in glob.glob('data/Sales/*.csv')])
print(f"Temps écoulé avec Pandas : {(time.time() - cur_time)}")
print(df.head(10))

Temps écoulé avec Pandas : 0.8800783157348633
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1      NaN                         NaN              NaN        NaN   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1             NaN                                     NaN  
2  

In [20]:
# Assignation d'un minuteur
cur_time = time.time()

In [21]:
# Concaténation de tous les fichiers CSV récupérés avec duckDB converti en df pandas 😮
# SELECT * : tout sélectionner
# FROM read_csv_auto(ref1, ref2, ref3, ref4) :
    # ref1 : chemin et nom du fichier (*.csv -> récupérer tous les fichiers .csv)
    # ref2 : header = True -> désigner la 1ère ligne en tant qu'en-tête des colonnes
    # ref3 : filename = True -> ???
    # ref4 : columns = {'Champ1' : INTEGER, 'Champ2' :...} type par colonne mais avec read_csv_auto pas nécessaire de faire ça
# LIMIT INT : nombre de lignes à sélectionner
df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/Sales/*.csv', header=True)
    LIMIT 10
    """).df() # .df() -> conversion en df pandas
print(f"Temps écoulé avec DuckDB : {(time.time() - cur_time)}")
print(df)

Temps écoulé avec DuckDB : 0.24290895462036133
  Order ID                     Product Quantity Ordered Price Each  \
0   176558        USB-C Charging Cable                2      11.95   
1      NaN                         NaN              NaN        NaN   
2   176559  Bose SoundSport Headphones                1      99.99   
3   176560                Google Phone                1        600   
4   176560            Wired Headphones                1      11.99   
5   176561            Wired Headphones                1      11.99   
6   176562        USB-C Charging Cable                1      11.95   
7   176563  Bose SoundSport Headphones                1      99.99   
8   176564        USB-C Charging Cable                1      11.95   
9   176565          Macbook Pro Laptop                1       1700   

       Order Date                        Purchase Address  
0  04/19/19 08:46            917 1st St, Dallas, TX 75001  
1             NaN                                     NaN  
2 

In [22]:
# Création d'une table virtuelle pour effectuer les instructions ci-après
conn.register("df_view", df)

In [23]:
# Fonction native describe() équivalent à Pandas
conn.execute("DESCRIBE df_view").df()

,column_name,column_type,null,key,default,extra
0,Order ID,VARCHAR,YES,NaN,NaN,NaN
1,Product,VARCHAR,YES,NaN,NaN,NaN
2,Quantity Ordered,VARCHAR,YES,NaN,NaN,NaN
3,Price Each,VARCHAR,YES,NaN,NaN,NaN
4,Order Date,VARCHAR,YES,NaN,NaN,NaN
5,Purchase Address,VARCHAR,YES,NaN,NaN,NaN


In [24]:
# Nouvelle DF sans limite de lignes cette fois-ci
df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/Sales/*.csv', header=True)
    """).df()

# Création d'une table virtuelle pour effectuer les instructions ci-après
conn.register("df_view", df)

In [25]:
# Nombre de lignes
conn.execute("SELECT COUNT(*) FROM df_view").df()

,count_star()
0,186850


In [26]:
# Suppression des lignes avec pandas car avec duckDB c'est plus compliqué...
# Nombre de lignes vides
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [27]:
# Suppression des lignes vides
df = df.dropna(how='all')

In [28]:
# Confirmation qu'il n'y a plus de lignes vides
df.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [29]:
# Nombre de lignes avec la DF pandas rectifiée
conn.execute("SELECT COUNT(*) FROM df").df()

,count_star()
0,186305


In [38]:
# Condition : respecter toujours les "" et les '' comme mentionnées ci-après !!!
conn.execute(
    """SELECT * FROM df 
    WHERE "Order ID" = '295665'""").df()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"


In [51]:
# Nouvelle table en copiant la table initiale et modifiant les colonnes :
# "NomColonneInitiale"::type AS NouveauNomColonne,
# "NomColonneInitiale" AS NouveauNomColonne,
conn.execute(
    """
    CREATE OR REPLACE TABLE sales AS
        SELECT
            "Order ID"::INTEGER AS order_id,
            "Product" AS product,
            "Quantity Ordered"::INTEGER AS quantity,
            "Price Each"::DECIMAL AS price_each,
            strptime("Order Date", '%m/%d/%Y %H:%M')::DATE as order_date,
            "Purchase Address" AS purchase_address
        FROM df
        WHERE
            TRY_CAST("Order ID" AS INTEGER) NOTNULL
    """)

# La fonction TRY_CAST... permet de supprimer les doublons des en-têtes
# qui sont apparus en concaténant tous les fichiers mensuels

In [54]:
# Affichage de la nouvelle DF... pb avec la colonne date non résolue
conn.execute("FROM sales").df()

ConversionException: Conversion Error: Could not convert DATE (0019-04-19) to nanoseconds

In [55]:
# Exclure certaines colonnes à l'affichage
conn.execute("""
	SELECT 
		* EXCLUDE (product, order_date, purchase_address)
	FROM sales
	""").df()

,order_id,quantity,price_each
0,176558,2,11.95
1,176559,1,99.99
2,176560,1,600.00
3,176560,1,11.99
4,176561,1,11.99
...,...,...,...
185945,259353,3,2.99
185946,259354,1,700.00
185947,259355,1,700.00
185948,259356,1,379.99


In [57]:
# Affichage de la valeur minimum de certaines colonnes
# Pour avoir d'autres conditions : 
# https://duckdb.org/2022/05/04/friendlier-sql.html
conn.execute(
    """
    SELECT
        MIN(COLUMNS(* EXCLUDE (product, order_date, purchase_address)))
    FROM sales 
    """).df()

,min(sales.order_id),min(sales.quantity),min(sales.price_each)
0,141234,1,2.99


In [61]:
# TCD par la colonne 'order_id' et en modifiant les données des colonnes : 
# On modifie cette fois-ci les données des colonnes, autres que 
# par le type et le nom d'en-tête de la colonne, donc on a recours à
# l'instruction "CREATE OR REPLACE VIEW" au lieu de "CREATE OR REPLACE TABLE"
conn.execute(
    """
    CREATE OR REPLACE VIEW aggregated_sales AS
    SELECT
        order_id, 
        COUNT(1) as nb_orders,
        MONTH(order_date) as month,
        str_split(purchase_address, ',')[2] AS city,
        SUM(quantity * price_each) AS revenue
    FROM sales
    GROUP BY ALL
    """)

In [62]:
conn.execute("FROM aggregated_sales").df()

,order_id,nb_orders,month,city,revenue
0,220485,1,6,Los Angeles,14.95
1,220486,1,6,San Francisco,149.99
2,220490,1,6,San Francisco,400.00
3,220495,1,6,San Francisco,999.99
4,220498,1,6,Boston,11.99
...,...,...,...,...,...
178432,233435,1,7,Los Angeles,3.84
178433,231783,1,7,Los Angeles,300.00
178434,215117,1,6,San Francisco,11.95
178435,211058,2,6,Austin,1850.00


In [63]:
# Revenues par ville par ordre décroissant (DESC)
conn.execute(
    """
    SELECT
        city,
        SUM(revenue) as total
    FROM aggregated_sales
    GROUP BY city
    ORDER BY total DESC
    """).df()

,city,total
0,San Francisco,8262203.91
1,Los Angeles,5452570.80
2,New York City,4664317.43
3,Boston,3661642.01
4,Atlanta,2795498.58
5,Dallas,2767975.40
6,Seattle,2747755.48
7,Portland,2320490.61
8,Austin,1819581.75


In [70]:
# Conversion de la DF aggregated_sales en fichier .parquet
# conn.execute(
#     """
#     COPY (FROM aggregated_sales) 
#     TO 'data/Sales/aggregated_sales.parquet' 
#     (FORMAT 'parquet')
#     """)

In [71]:
# Récupération du fichier .parquet
conn.execute(
    """
    FROM 'data/Sales/aggregated_sales.parquet'
    """).df()

,order_id,nb_orders,month,city,revenue
0,220485,1,6,Los Angeles,14.95
1,220486,1,6,San Francisco,149.99
2,220490,1,6,San Francisco,400.00
3,220495,1,6,San Francisco,999.99
4,220498,1,6,Boston,11.99
...,...,...,...,...,...
178432,233435,1,7,Los Angeles,3.84
178433,231783,1,7,Los Angeles,300.00
178434,215117,1,6,San Francisco,11.95
178435,211058,2,6,Austin,1850.00
